In [2]:
import random
import itertools
from decimal import Decimal
import os
from datetime import datetime
import csv

In [3]:
probabilities = {
    13: (100.00, 7.69),
    12: (92.31, 15.38),
    11: (84.62, 23.08),
    10: (76.92, 30.77),
    9: (69.23, 38.46),
    8: (61.54, 46.15),
    7: (53.85, 53.85),
    6: (46.15, 61.54),
    5: (38.46, 69.23),
    4: (30.77, 76.92),
    3: (23.08, 84.62),
    2: (15.38, 92.31),
    1: (7.69, 100.00),
}

In [4]:
payouts = {
    13: (1.00, 2.00),
    12: (1.15, 1.92),
    11: (1.23, 1.84),
    10: (1.30, 1.76),
    9: (1.38, 1.69),
    8: (1.46, 1.61),
    7: (1.53, 1.53),
    6: (1.61, 1.46),
    5: (1.69, 1.38),
    4: (1.76, 1.30),
    3: (1.84, 1.23),
    2: (1.92, 1.15),
    1: (2.00, 1.00),
}

In [5]:
LOWER = 0
HIGHER = 1

In [6]:
def is_degen(degen_rate: int) -> bool:
    return random.randint(1, 100) <= degen_rate

In [7]:
is_degen(50)

True

In [8]:
def pick_card() -> int:
    return random.randint(1,  13)

In [9]:
pick_card()

12

In [10]:
DEFAULT_BET_AMOUNT_OPTIONS = [1, 2, 5, 10, 20]
DEFAULT_BET_AMOUNT_PROBABILITIES = [20, 30, 30, 15, 5]

In [11]:
DEFAULT_BET_OPTIONS = list(itertools.chain(*[(x,) * y for x, y in zip(DEFAULT_BET_AMOUNT_OPTIONS, DEFAULT_BET_AMOUNT_PROBABILITIES)]))

In [12]:
DEFAULT_SUPPORTERS_PAYOUT = 0.04
DEFAULT_TEAM_PAYOUT = 0.01

In [13]:
DEFAULT_STARTING_FUNDS = 5000

In [14]:
FIELDNAMES = [
    "round",
    "starting_pool_amount",
    "degen",
    "first_card",
    "bet_amount",
    "usable_bet_amount",
    "bet_high_or_low_first",
    "second_card",
    "intermediate_win",
    "payout_coefficient_first",
    "bet_high_or_low_second",
    "third_card",
    "win",
    "payout_coefficient_second",
    "combined_payout_coefficient",
    "payout_amount",
    "updated_pool_amount",
    "cumulative_payout",
    "to_supporters",
    "cumulative_to_supporters",
    "to_team",
    "cumulative_to_team",
]

In [15]:
CONFIG_FIELDS = [
    "starting_pool_amount",
    "degen_rate",
    "supporters_payout",
    "team_payout",
    "bet_amounts",
    "bet_amount_probabilities",
]

In [16]:
def check_win(card1, card2, bet):
    win = False

    if bet == HIGHER:
        if card1 == 1:
            if card2 > card1:
                win = True;
            
        elif card1 == 13:
            if card2 == card1:
                win = True;
            
        else:
            if card2 >= card1:
                win = True;


    else:
        if card1 == 1:
            if card2 == card1:
                win = True;
            
        elif card1 == 13:
            if card2 < card1:
                win = True;
            
        else:
            if card2 <= card1:
                win = True;

    return win


In [17]:
check_win(5, 8, 1)

True

In [18]:
_TWO_DECIMAL_QUANTIZER = Decimal("0.01")
_FOUR_DECIMAL_QUANTIZER = Decimal("0.0001")

In [19]:
def run_simulation(
    number_of_rounds,
    degen_rate,
    starting_pool_amount=DEFAULT_STARTING_FUNDS,
    supporters_payout=DEFAULT_SUPPORTERS_PAYOUT,
    team_payout=DEFAULT_TEAM_PAYOUT,
    bet_amounts=None,
    bet_amount_probabilities=None
):
    bet_amounts = bet_amounts or DEFAULT_BET_AMOUNT_OPTIONS
    bet_amount_probabilities = bet_amount_probabilities or DEFAULT_BET_AMOUNT_PROBABILITIES
    bet_options = list(
        itertools.chain(*[(x,) * y for x, y in zip(bet_amounts, bet_amount_probabilities)])
    )

    pool_size = starting_pool_amount
    cumulative_payout = Decimal(0)
    cumulative_to_supporters = Decimal(0)
    cumulative_to_team = Decimal(0)
    
    dir_name = f'reports/simulation_{datetime.now().isoformat()}'
    os.mkdir(dir_name)
    
    with open(f'{dir_name}/config.txt', 'w') as f:
        f.write(f"""Starting pool size: {starting_pool_amount}
Degen rate: {degen_rate}
Supporters payout percentage: {supporters_payout}
Team payout percentage: {team_payout}
Bet amounts: {bet_amounts}
Bet amount probabilities: {bet_amount_probabilities}""")
    
    f = open(f'{dir_name}/simulation.csv', 'w')
    outwriter = csv.DictWriter(f, fieldnames=FIELDNAMES)
    outwriter.writeheader()
    
    for i in range(1, number_of_rounds + 1):
        if i % 1000 == 0:
            print(f"Round {i}")
            
        initial_pool_size = pool_size
        payout_coefficient_first = bet_second = third_card = win = payout_coefficient_second = combined_payout_coefficient = payout_amount = ""
        degen = is_degen(degen_rate)
        bet_amount = bet_options[random.randint(0, 99)]
        to_supporters = Decimal(bet_amount * supporters_payout).quantize(_TWO_DECIMAL_QUANTIZER)
        cumulative_to_supporters += to_supporters
        to_team = Decimal(bet_amount * team_payout).quantize(_TWO_DECIMAL_QUANTIZER)
        cumulative_to_team += to_team
        usable_bet_amount = bet_amount - to_supporters - to_team

        first_card = pick_card()
        if degen:
            bet_first = LOWER if first_card <= 7 else HIGHER
        else:
            bet_first = HIGHER if first_card <= 7 else LOWER
            
        second_card = pick_card()
        intermediate_win = check_win(first_card, second_card, bet_first)
        if intermediate_win:
            payout_coefficient_first = payouts[first_card][bet_first]
            
            if degen:
                bet_second = LOWER if second_card <= 7 else HIGHER
            else:
                bet_second = HIGHER if second_card <= 7 else LOWER
            
            third_card = pick_card()
            win = check_win(second_card, third_card, bet_second)
            if win:
                payout_coefficient_second = payouts[second_card][bet_second]
                combined_payout_coefficient = Decimal(payout_coefficient_first * payout_coefficient_second).quantize(_FOUR_DECIMAL_QUANTIZER)
                payout_amount = Decimal(usable_bet_amount * combined_payout_coefficient).quantize(_FOUR_DECIMAL_QUANTIZER)
                pool_size -= (payout_amount - usable_bet_amount)
                cumulative_payout += (payout_amount - usable_bet_amount)
            else:
                pool_size += usable_bet_amount
                
        else:
            pool_size += usable_bet_amount
            
        data = {    
            "round": i,
            "starting_pool_amount": initial_pool_size,
            "degen": degen,
            "first_card": first_card,
            "bet_amount": bet_amount,
            "usable_bet_amount": usable_bet_amount,
            "bet_high_or_low_first": bet_first,
            "second_card": second_card,
            "intermediate_win": intermediate_win,
            "payout_coefficient_first": payout_coefficient_first,
            "bet_high_or_low_second": bet_second,
            "third_card": third_card,
            "win": win,
            "payout_coefficient_second": payout_coefficient_second,
            "combined_payout_coefficient": combined_payout_coefficient,
            "payout_amount": payout_amount,
            "updated_pool_amount": pool_size,
            "cumulative_payout": cumulative_payout,
            "to_supporters": to_supporters,
            "cumulative_to_supporters": cumulative_to_supporters,
            "to_team": to_team,
            "cumulative_to_team": cumulative_to_team,
        }
        outwriter.writerow(data)
        
    f.close()


In [20]:
run_simulation(10000, 0, starting_pool_amount=500)

Round 1000
Round 2000
Round 3000
Round 4000
Round 5000
Round 6000
Round 7000
Round 8000
Round 9000
Round 10000
